### Exercise 1-2. 
The steel model of this chapter can be further modified to reflect various changes in production requirements. For each part below, explain the modifications to Figures 1-6a and 1-6b that
would be required to achieve the desired changes. (Make each change separately, rather than accumulating the changes from one part to the next.)

(a) How would you change the constraints so that total hours used by all products must equal the total hours available for each stage? Solve the linear program with this change, and verify that you get the same results. Explain why, in this case, there is no difference in the solution.

(b) How would you add to the model to restrict the total weight of all products to be less than a new parameter, max_weight? Solve the linear program for a weight limit of 6500 tons, and explain how this extra restriction changes the results.

(c) The incentive system for mill managers may tend to encourage them to produce as many tons as possible. How would you change the objective function to maximize total tons? For the data of our example, does this make a difference to the optimal solution?
    
(d) Suppose that instead of the lower bounds represented by commit[p] in our model, we want to require that each product represent a certain share of the total tons produced. In the algebraic notation of Figure 1-1, this new constraint might be represented as 
    
\begin{equation*}
X_j >= s_j \thinspace \sum_{k \thinspace \epsilon \thinspace P} X_k \thinspace , for \thinspace each \thinspace  j \thinspace \epsilon \thinspace P
\end{equation*}
        
where sj is the minimum share associated with project j. How would you change the AMPL model to use this constraint in place of the lower bounds commit[p]? If the minimum shares are 0.4 for bands and plate, and 0.1 for coils, what is the solution ? Verify that if you change the minimum shares to 0.5 for bands and plate, and 0.1 for coils, the linear program gives an optimal solution that produces nothing, at zero profit. Explain why this makes sense.
      
(e) Suppose there is an additional finishing stage for plates only, with a capacity of 20 hours and a rate of 150 tons per hour. Explain how you could modify the data, without changing the model, to incorporate this new stage.


In [1]:
import pandas as pd
from amplpy import AMPL, Environment

In [2]:
ampl = AMPL(Environment('/opt/ampl.linux64'))

In [3]:
ampl.reset()

In [4]:
ampl.read('./ex--1-2a.mod')

In [5]:
ampl.readData('./ex--1-2a.dat')

In [63]:
print(ampl.getObjective('audience_coverage').get())

maximize audience_coverage:
	1800000*X['number_of_minutes_in_TV'] + 
	1e+06*X['number_of_pages_in_magazine'] + 
	250000*X['number_of_minutes_in_radio'];


In [64]:
print(ampl.getConstraint('constraints'))
print(ampl.getConstraint('constraints').get('budget'))
print(ampl.getConstraint('constraints').get('person-week'))

subject to constraints{c in constraint_names} : sum{p in
  advertising_platform} constraint_list[p,c]*X[p] <= constraint_limits[c];
subject to constraints['budget']:
	20000*X['number_of_minutes_in_TV'] + 
	10000*X['number_of_pages_in_magazine'] + 
	2000*X['number_of_minutes_in_radio'] <= 1e+06;
subject to constraints['person-week']:
	X['number_of_minutes_in_TV'] + 3*X['number_of_pages_in_magazine'] + 
	X['number_of_minutes_in_radio'] <= 100;


In [65]:
ampl.eval('expand audience_coverage;')
ampl.eval('expand constraints;')

maximize audience_coverage:
	1800000*X['number_of_minutes_in_TV'] + 
	1e+06*X['number_of_pages_in_magazine'] + 
	250000*X['number_of_minutes_in_radio'];

subject to constraints['budget']:
	20000*X['number_of_minutes_in_TV'] + 
	10000*X['number_of_pages_in_magazine'] + 
	2000*X['number_of_minutes_in_radio'] <= 1e+06;

subject to constraints['person-week']:
	X['number_of_minutes_in_TV'] + 3*X['number_of_pages_in_magazine'] + 
	X['number_of_minutes_in_radio'] <= 100;



In [66]:
ampl.eval('expand X["number_of_minutes_in_TV"];')
ampl.eval('expand X["number_of_pages_in_magazine"];')
ampl.eval('expand X["number_of_minutes_in_radio"];')

Coefficients of X['number_of_minutes_in_TV']:
	constraints['budget']         20000
	constraints['person-week']        1
	audience_coverage           1800000

Coefficients of X['number_of_pages_in_magazine']:
	constraints['budget']       10000
	constraints['person-week']      3
	audience_coverage           1e+06

Coefficients of X['number_of_minutes_in_radio']:
	constraints['budget']         2000
	constraints['person-week']       1
	audience_coverage           250000



In [67]:
ampl.eval('display _nvars, _ncons;')

_nvars = 3
_ncons = 2



In [68]:
ampl.solve()

MINOS 5.51: optimal solution found.
2 iterations, objective 93700000


In [69]:
print(ampl.getObjective('audience_coverage').get().value())

93699999.99999999


#### Shadow Price or Dual Price or Marginal Price

In [70]:
ampl.eval('display constraints;')
print(ampl.getConstraint('constraints').getValues())

constraints [*] :=
     budget     86.1111
person-week  77777.8
;

   index0    | constraints.dual
  'budget'   | 86.111111111111086
'person-week' | 77777.777777777839



### Decision Variable Solution and Reduced Cost

In [71]:
X_result = list()
items = ('number_of_minutes_in_TV',
        'number_of_pages_in_magazine',
        'number_of_minutes_in_radio')
for p in items:
    X_result.append([
        p,
        ampl.getVariable('X').get(p).lb(), 
        ampl.getVariable('X').get(p).value(),
        ampl.getVariable('X').get(p).ub(),
        ampl.getVariable('X').get(p).rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
number_of_minutes_in_TV,10.0,44.0,99999.0,3.783498e-10
number_of_pages_in_magazine,2.0,2.0,99999.0,-9.444444e+04
number_of_minutes_in_radio,0.0,50.0,120.0,0.000000e+00


In [72]:
print(ampl.getData('X'))
values = ampl.getVariable('X').getValues()
print(values)
values.toPandas()
print(values.toList())
print(values.toDict())

   index0    |      X      
'number_of_minutes_in_TV' | 43.999999999999993
'number_of_minutes_in_radio' | 49.999999999999993
'number_of_pages_in_magazine' |      2      

   index0    |    X.val    
'number_of_minutes_in_TV' | 43.999999999999993
'number_of_minutes_in_radio' | 49.999999999999993
'number_of_pages_in_magazine' |      2      

[('number_of_minutes_in_TV', 43.99999999999999), ('number_of_minutes_in_radio', 49.99999999999999), ('number_of_pages_in_magazine', 2.0)]
{'number_of_minutes_in_TV': 43.99999999999999, 'number_of_minutes_in_radio': 49.99999999999999, 'number_of_pages_in_magazine': 2.0}


## Summary

1. Adding radio as a medium for advertisement, improved objective cost function. 
2. Shadow price for budget constraint reduced from ~$\$$88 to ~$\$$86. Almost no impact to this constraint.  
3. Shadow price for person-week constraint increased significantly ~$\$$40,000 to ~$\$$77,777. Adding radio as a medium has now increased shadow price of person-week. Adding radio as a medium has increased the importance of person-week constraint.
4. Finally:
 * Once again, TV medium is no more at the minimum minutes once the person-week constraint was added. 
 * Magazine as a medium is not preferred anymore. Only radio and TV enters the solution, magazine is no more a pereferred medium. 
 * The person-week required for TV and radio is very less while for magazine is higher. Making it attractive to allocate all the budget between TV and radio only. This is also indicative of person-week constraint causing the magazine variable to leave the solution. 
5. Adding an upper bound of 120 radio minutes as no impact to the solution. The objective remains same as in 1-1c. The reduced costs and constraint shadow prices (dual prices) also has no change because the radio minutes variable (=50) is in between the min and max bounds. 
6. Adding the min pages for the magazine, 
    
    (a) reduced the objective cost function from $\$$93.88million to $\$$93.7million. 
    
    (b) Adding this min bound did not cause any change to the shadow price for either constraints. 
    
    (c) This means that at a min bound of 2 magazine pages, there is no additional impact to the constraints. 
    
    (d) The min bound of 2 magazine pages, did not impact reduced costs for all three variables.
    
    (e) An interesting observation is that, adding a min bound of 2 pages, made the solution for all variables an integer value. In the previous case (1-1c), the solutions were non-integers. 
7. A note here is that, TV minute and Magazine pages variables have no upper bounds. This was modelled by setting the upper bounds to 9999. 